In [127]:
import pandas as pd
import numpy as np
import pdb
import pymysql
import warnings
con = pymysql.connect (host ='10.200.2.80',
user ='analysis_rw02',
password = 'rU#W9UI6',
port=9030)

In [128]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus']=False
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 1.政府评分卡

In [76]:
sql_zf='''
select  CONCAT(YEAR(CURDATE()), "-", LPAD(MONTH(CURDATE()), 2, "0")) as "statis_month" ,ccx_code, ent_name, uscc ,reg_code, 
		case when xz_level = "国务院" then "A"
			 when xz_level = "省部级" then "B"
			 when xz_level = "司厅局级" then "C"
			 when xz_level = "县处级" then "D"
			 when xz_level = "乡镇科级" then "E"
			 else "F" END as "credit_level",
			 now() as "create_time",
			 e.status
		from 
		(
			select a.*, d.status from jdbc_mysql_seed.dws_seed_storage_db.dws_seed_storage_basic a
			left join
			(
			select *,
			   case when reg_status like "%注吊销%" or (reg_status like "%注销%" and reg_status not like "%未注销%" and reg_status not like '%未办理注销%') then "注销"
			        when reg_status like "%吊销%" then "吊销"
			        when reg_status like "%撤销%" or reg_status like "%撤消" then "撤销"
			        when reg_status like "%停业%" then "停业"
			        when reg_status like "%歇业%" then "歇业"
			        when reg_status like "%清算%" then "清算"
			        when reg_status like "%除名%" then "除名"
			        when reg_status like "%责令关闭%" then "责令关闭"
			        when reg_status like "%迁入%" then "迁入"
			        when reg_status like "%迁出%" or reg_status like "%迁出外省%" or reg_status like "%迁出市外%" then "迁出"
			        when reg_status like "%存续%" or reg_status like "%在营%" or reg_status like "%在业%" or reg_status like "%开业%" 
			        	or (reg_status like "%正常%" or reg_status not like "%非正常%") or reg_status like "%存在%" then "在续"
			        else reg_status end as "status"
				from 
					(
						select * from
							(
								select *, ROW_NUMBER() over(PARTITION by ccx_code order by update_time desc) as "rnk"
								from ccx_ent_info_db.basic
								where deleted = "0"
							)b
						where rnk = 1 and up_state != "3"
					) c
				where deleted = "0"
			)d
			on a.ccx_code = d.ccx_code
		) e
		where reg_code = "C01";
'''
zf_card = pd.read_sql(sql_zf,con)


In [77]:
zf_card.shape

(273517, 8)

In [78]:
zf_card.head()

,statis_month,ccx_code,ent_name,uscc,reg_code,credit_level,create_time,status
0,2024-07,CCX21635644920036L,中国共产党中江县龙台镇委员会,115105210083947450,C01,E,2024-07-22 11:23:53,None
1,2024-07,CCX21635644920182g,普宁市下架山镇人民政府,11445281007033374N,C01,E,2024-07-22 11:23:53,None
2,2024-07,CCX238950626226VIj,泊头市水务局,11130981000989402T,C01,D,2024-07-22 11:23:53,None
3,2024-07,CCX27774373654993N,三亚市教育局,114602000082323598,C01,C,2024-07-22 11:23:53,None
4,2024-07,CCX27774373655017V,上思县财政局,11450621007714292L,C01,D,2024-07-22 11:23:53,None


# 2.央企评分卡

In [79]:
sql_yq='''
    select CONCAT(YEAR(CURDATE()), '-', LPAD(MONTH(CURDATE()), 2, '0')) as 'statis_month',
    	   ccx_code, ent_name, uscc ,reg_code, 'A' as 'eva_results', now() as 'create_time', e.status 
    from 
    (
    	select a.*, d.status from jdbc_mysql_seed.dws_seed_storage_db.dws_seed_storage_basic a
    	left join
    	(
    	select *,
    	   case when reg_status like '%注吊销%' or (reg_status like '%注销%' and reg_status not like '%未注销%' and reg_status not like '%未办理注销%') then '注销'
    	        when reg_status like '%吊销%' then '吊销'
    	        when reg_status like '%撤销%' or reg_status like '%撤消' then '撤销'
    	        when reg_status like '%停业%' then '停业'
    	        when reg_status like '%歇业%' then '歇业'
    	        when reg_status like '%清算%' then '清算'
    	        when reg_status like '%除名%' then '除名'
    	        when reg_status like '%责令关闭%' then '责令关闭'
    	        when reg_status like '%迁入%' then '迁入'
    	        when reg_status like '%迁出%' or reg_status like '%迁出外省%' or reg_status like '%迁出市外%' then '迁出'
    	        when reg_status like '%存续%' or reg_status like '%在营%' or reg_status like '%在业%' or reg_status like '%开业%' 
    	        	or (reg_status like '%正常%' or reg_status not like '%非正常%') or reg_status like '%存在%' then '在续'
    	        else reg_status end as 'status'
    		from 
    			(
    				select * from
    					(
    						select *, ROW_NUMBER() over(PARTITION by ccx_code order by update_time desc) as 'rnk'
    						from ccx_ent_info_db.basic
    						where deleted = '0' 
    					)b
    				where rnk = 1 and up_state != '3'
    			) c
    		where deleted = '0'
    	)d
    	on a.ccx_code = d.ccx_code
    ) e
    where reg_code = 'C02'
'''
yq_card = pd.read_sql(sql_yq,con)

In [80]:
yq_card.shape

(101, 8)

In [81]:
yq_card.head()

,statis_month,ccx_code,ent_name,uscc,reg_code,eva_results,create_time,status
0,2024-07,CCX326678695606Jdu,东风汽车集团有限公司,914200001000115161,C02,A,2024-07-22 11:24:23,在续
1,2024-07,CCX890531649983ttm,华润（集团）有限公司,None,C02,A,2024-07-22 11:24:23,None
2,2024-07,CCX237767832584qSO,中国远洋海运集团有限公司,91310000MA1FL1MMXL,C02,A,2024-07-22 11:24:23,在续
3,2024-07,CCX765381997779UwL,中国机械科学研究总院集团有限公司,91110000400008060U,C02,A,2024-07-22 11:24:23,在续
4,2024-07,CCX267450928742HHp,中国林业集团有限公司,911100001000016682,C02,A,2024-07-22 11:24:23,在续


# 3.民企评分卡

In [82]:
sql_mq='''
select CONCAT(YEAR(CURDATE()), '-', LPAD(MONTH(CURDATE()), 2, '0')) as 'statis_month',
	   ccx_code, ent_name, uscc ,reg_code,
	   case when cast(tag as int) > 0 and cast(tag as int) <= 20 then 'A'
	   		when cast(tag as int) > 20 and cast(tag as int) <= 50 then 'B'
	   		when cast(tag as int) > 50 and cast(tag as int) <= 100 then 'C'
	   		when cast(tag as int) > 100 and cast(tag as int) <= 200 then 'D'
	   		when cast(tag as int) > 200 and cast(tag as int) <= 300 then 'E'
	   		else 'F' end as 'eva_results',
	   	now() as 'create_time',
	   	e.status
	   	from
	   	(
	   		select a.*, d.status from jdbc_mysql_seed.dws_seed_storage_db.dws_seed_storage_basic a
			left join
			(
			select *,
			   case when reg_status like '%注吊销%' or (reg_status like '%注销%' and reg_status not like '%未注销%' and reg_status not like '%未办理注销%') then '注销'
			        when reg_status like '%吊销%' then '吊销'
			        when reg_status like '%撤销%' or reg_status like '%撤消' then '撤销'
			        when reg_status like '%停业%' then '停业'
			        when reg_status like '%歇业%' then '歇业'
			        when reg_status like '%清算%' then '清算'
			        when reg_status like '%除名%' then '除名'
			        when reg_status like '%责令关闭%' then '责令关闭'
			        when reg_status like '%迁入%' then '迁入'
			        when reg_status like '%迁出%' or reg_status like '%迁出外省%' or reg_status like '%迁出市外%' then '迁出'
			        when reg_status like '%存续%' or reg_status like '%在营%' or reg_status like '%在业%' or reg_status like '%开业%' 
			        	or (reg_status like '%正常%' or reg_status not like '%非正常%') or reg_status like '%存在%' then '在续'
			        else reg_status end as 'status'
				from 
					(
						select * from
							(
								select *, ROW_NUMBER() over(PARTITION by ccx_code order by update_time desc) as 'rnk'
								from ccx_ent_info_db.basic
								where deleted = '0' 
							)b
						where rnk = 1 and up_state != '3'
					) c
				where deleted = '0'
			)d
			on a.ccx_code = d.ccx_code
	   	)e
	   	where reg_code = 'C08'
'''
mq_card = pd.read_sql(sql_mq,con)

In [83]:
mq_card.shape

(1341, 8)

In [84]:
mq_card.head()

,statis_month,ccx_code,ent_name,uscc,reg_code,eva_results,create_time,status
0,2024-07,CCX257405568576eEO,江苏雅克科技股份有限公司,91320200250268472W,C08,F,2024-07-22 11:25:04,在续
1,2024-07,CCX262895911632BPu,四川省商业投资集团有限责任公司,915100002018168664,C08,F,2024-07-22 11:25:04,在续
2,2024-07,CCX610100426016UiP,宁波均胜电子股份有限公司,9133020060543096X6,C08,E,2024-07-22 11:25:04,在续
3,2024-07,CCX629637494964kix,山东玲珑轮胎股份有限公司,91370000613418880Y,C08,F,2024-07-22 11:25:04,在续
4,2024-07,CCX734620973825loW,广汇汽车服务集团股份公司,912102003411090040,C08,D,2024-07-22 11:25:04,在续


# 4. 国企评分卡

## 4.1地区经济评分卡相关指标—不用加工

In [85]:
sql_dqjj_index='''
select CONCAT(YEAR(CURDATE()), '-', LPAD(MONTH(CURDATE()), 2, '0')) as 'statis_month',
	   e.ccx_code, e.ent_name, e.uscc , e.reg_code, 
	   e.地区生产总值, e.人均地区生产总值, e.地区经济发展趋势, e.地方公共财政收入, e.地区财政收入质量, e.地区财政依存度, e.地区土地财政依赖度, e.地区财政自给率, e.地区负债率, e.地区债务率,
	   now() as 'create_time',
	   h.status 
from 
(
	select * from jdbc_mysql_seed.dws_seed_storage_db.dws_seed_storage_basic a
	left join 
	(
		select public_year, national, province, city,
		min(case when norm_type = '地区生产总值' then cast( norm_value as float ) else null end) as '地区生产总值',
		min(case when norm_type = '人均地区生产总值' then cast( norm_value as float ) else null end) as '人均地区生产总值',
		min(case when norm_type = '地区经济发展趋势' then cast( norm_value as float ) else null end) as '地区经济发展趋势',
		min(case when norm_type = '地方公共财政收入' then cast( norm_value as float ) else null end) as '地方公共财政收入',
		min(case when norm_type = '地区财政收入质量' then cast( norm_value as float ) else null end) as '地区财政收入质量',
		min(case when norm_type = '地区财政依存度' then cast( norm_value as float ) else null end) as '地区财政依存度',
		min(case when norm_type = '地区土地财政依赖度' then cast( norm_value as float ) else null end) as '地区土地财政依赖度',
		min(case when norm_type = '地区财政自给率' then cast( norm_value as float ) else null end) as '地区财政自给率',
		min(case when norm_type = '地区负债率' then cast( norm_value as float ) else null end) as '地区负债率',
		min(case when norm_type = '地区债务率' then cast( norm_value as float ) else null end) as '地区债务率'
		from
		(
			select * from
				(
					select *, ROW_NUMBER() over(PARTITION by public_year, national, province, city, norm_type order by update_time desc) as 'rnk'
					from jdbc_mysql_seed.dws_seed_storage_db.dws_area_norm
					where deleted = '0' and up_state != '3' and city in ('', '北京市', '天津市', '上海市', '重庆市')
				)a
				where rnk = 1 and public_year = '2023'
		)b
		where deleted = '0' 
		group by public_year, national, province, city
		order by public_year
	)c on a.province = c.province
	where reg_code = 'C03'
	UNION
	select * from jdbc_mysql_seed.dws_seed_storage_db.dws_seed_storage_basic a
	left join 
	(
		select public_year, national, province, city,
		min(case when norm_type = '地区生产总值' then cast( norm_value as float ) else null end) as '地区生产总值',
		min(case when norm_type = '人均地区生产总值' then cast( norm_value as float ) else null end) as '人均地区生产总值',
		min(case when norm_type = '地区经济发展趋势' then cast( norm_value as float ) else null end) as '地区经济发展趋势',
		min(case when norm_type = '地方公共财政收入' then cast( norm_value as float ) else null end) as '地方公共财政收入',
		min(case when norm_type = '地区财政收入质量' then cast( norm_value as float ) else null end) as '地区财政收入质量',
		min(case when norm_type = '地区财政依存度' then cast( norm_value as float ) else null end) as '地区财政依存度',
		min(case when norm_type = '地区土地财政依赖度' then cast( norm_value as float ) else null end) as '地区土地财政依赖度',
		min(case when norm_type = '地区财政自给率' then cast( norm_value as float ) else null end) as '地区财政自给率',
		min(case when norm_type = '地区负债率' then cast( norm_value as float ) else null end) as '地区负债率',
		min(case when norm_type = '地区债务率' then cast( norm_value as float ) else null end) as '地区债务率'
		from
		(
			select * from
				(
					select *, ROW_NUMBER() over(PARTITION by public_year, national, province, city, norm_type order by update_time desc) as 'rnk'
					from jdbc_mysql_seed.dws_seed_storage_db.dws_area_norm
					where deleted = '0' and up_state != '3' and city != ''
				)b
				where rnk = 1 and public_year = '2023'
		)c
		where deleted = '0' 
		group by public_year, national, province, city
		order by public_year
	)d on a.city = d.city
	where reg_code = 'C04'
) e
left JOIN 
(
   select *,
   case when reg_status like '%注吊销%' or (reg_status like '%注销%' and reg_status not like '%未注销%' and reg_status not like '%未办理注销%') then '注销'
        when reg_status like '%吊销%' then '吊销'
        when reg_status like '%撤销%' or reg_status like '%撤消' then '撤销'
        when reg_status like '%停业%' then '停业'
        when reg_status like '%歇业%' then '歇业'
        when reg_status like '%清算%' then '清算'
        when reg_status like '%除名%' then '除名'
        when reg_status like '%责令关闭%' then '责令关闭'
        when reg_status like '%迁入%' then '迁入'
        when reg_status like '%迁出%' or reg_status like '%迁出外省%' or reg_status like '%迁出市外%' then '迁出'
        when reg_status like '%存续%' or reg_status like '%在营%' or reg_status like '%在业%' or reg_status like '%开业%' 
        	or (reg_status like '%正常%' or reg_status not like '%非正常%') or reg_status like '%存在%' then '在续'
        else reg_status end as 'status'
	from 
	(
		select * from
			(
				select *, ROW_NUMBER() over(PARTITION by ccx_code order by update_time desc) as 'rnk'
				from ccx_ent_info_db.basic
				where deleted = '0' 
			)f
		where rnk = 1 and up_state != '3'
	) g
	where deleted = '0'
) h
on e.ccx_code = h.ccx_code
'''
dqjj_index = pd.read_sql(sql_dqjj_index,con)

In [86]:
dqjj_index.shape

(15206, 17)

In [87]:
dqjj_index.columns

Index(['statis_month', 'ccx_code', 'ent_name', 'uscc', 'reg_code', '地区生产总值',
       '人均地区生产总值', '地区经济发展趋势', '地方公共财政收入', '地区财政收入质量', '地区财政依存度', '地区土地财政依赖度',
       '地区财政自给率', '地区负债率', '地区债务率', 'create_time', 'status'],
      dtype='object')

In [88]:
dqjj_index.head()

,statis_month,ccx_code,ent_name,uscc,reg_code,地区生产总值,人均地区生产总值,地区经济发展趋势,地方公共财政收入,地区财政收入质量,地区财政依存度,地区土地财政依赖度,地区财政自给率,地区负债率,地区债务率,create_time,status
0,2024-07,CCX233317672500gln,阳泉市南庄煤炭集团有限责任公司,911400007281648765,C04,1002.70,76830.0,103.2,83.10,NaN,0.082876,NaN,0.492590,NaN,NaN,2024-07-22 11:25:22,在续
1,2024-07,CCX237885272242GqI,大石桥市富桥实业有限公司,912108823188334671,C04,1479.40,64999.0,105.3,137.10,NaN,0.092673,NaN,0.541683,NaN,NaN,2024-07-22 11:25:22,在续
2,2024-07,CCX238552168157tyO,营口辽河老街旅游开发有限公司,91210800696188645A,C04,1479.40,64999.0,105.3,137.10,NaN,0.092673,NaN,0.541683,NaN,NaN,2024-07-22 11:25:22,在续
3,2024-07,CCX239038706901cQY,阳泉市汇通小额贷款股份有限公司,91140300689874671T,C04,1002.70,76830.0,103.2,83.10,NaN,0.082876,NaN,0.492590,NaN,NaN,2024-07-22 11:25:22,在续
4,2024-07,CCX239290367674TQQ,莆田市临港产业园投资开发有限公司,91350300691945942G,C04,3070.73,96291.0,103.6,162.81,NaN,0.053020,NaN,0.606233,0.387351,NaN,2024-07-22 11:25:22,在续


## 4.2 地方融资平台评分卡相关指标—需要加工

### 4.2.1 区域得分

In [130]:
sql_dfrz_qy_index = '''
select CONCAT(YEAR(CURDATE()), '-', LPAD(MONTH(CURDATE()), 2, '0')) as 'statis_month',
	   A.ccx_code, A.ent_name, A.uscc , A.reg_code, 
	   A.地方公共财政收入, A.地区债务率, A.地区偿债率, A.地区财政自给率, A.地区财政依存度, A.地区财政收入质量, A.地区生产总值, A.人均地区生产总值, A.地区第三产业占比, A.人均国内生产总值, A.地区常住人口, 
	   A.地方公共财政收入_2020, A.地方公共财政收入_2021, A.地方公共财政收入_2022, A.地方公共财政收入_2023,
	   A.地区生产总值_2020, A.地区生产总值_2021, A.地区生产总值_2022, A.地区生产总值_2023,
       A.地区常住人口_2020, A.地区常住人口_2021, A.地区常住人口_2022, A.地区常住人口_2023,
	   now() as 'create_time',
	   B.status 
from 
(
	select * from jdbc_mysql_seed.dws_seed_storage_db.dws_seed_storage_basic a
	left join 
	(
		select d.*, h.地方公共财政收入_2020, h.地方公共财政收入_2021, h.地方公共财政收入_2022, h.地方公共财政收入_2023,
			        h.地区生产总值_2020, h.地区生产总值_2021, h.地区生产总值_2022, h.地区生产总值_2023,
			        h.地区常住人口_2020, h.地区常住人口_2021, h.地区常住人口_2022, h.地区常住人口_2023 
		FROM 
		(
			select public_year, national, province, city,
			-- min(case when norm_type = '地区内城投有息负债' then cast( norm_value as float ) else null end) as '地区内城投有息负债',   在平台得分指标里面
			min(case when norm_type = '地方公共财政收入' then cast( norm_value as float ) else null end) as '地方公共财政收入',
			min(case when norm_type = '地区债务率' then cast( norm_value as float ) else null end) as '地区债务率',
			min(case when norm_type = '地区偿债率' then cast( norm_value as float ) else null end) as '地区偿债率',
			min(case when norm_type = '地区财政自给率' then cast( norm_value as float ) else null end) as '地区财政自给率',
			min(case when norm_type = '地区财政依存度' then cast( norm_value as float ) else null end) as '地区财政依存度',
			min(case when norm_type = '地区财政收入质量' then cast( norm_value as float ) else null end) as '地区财政收入质量',
			min(case when norm_type = '地区生产总值' then cast( norm_value as float ) else null end) as '地区生产总值',
			min(case when norm_type = '人均地区生产总值' then cast( norm_value as float ) else null end) as '人均地区生产总值',
			min(case when norm_type = '地区第三产业占比' then cast( norm_value as float ) else null end) as '地区第三产业占比',
			(select cast((select norm_value from jdbc_mysql_seed.dws_seed_storage_db.dws_area_norm where norm_type = '人均国内生产总值' and public_year = '2023') as float)) as '人均国内生产总值',
			min(case when norm_type = '地区常住人口' then cast( norm_value as float ) else null end) as '地区常住人口'
			from
			(
				select * from
					(
						select *, ROW_NUMBER() over(PARTITION by public_year, national, province, city, norm_type order by update_time desc) as 'rnk'
						from jdbc_mysql_seed.dws_seed_storage_db.dws_area_norm
						where deleted = '0' and up_state != '3' and city in ('', '北京市', '天津市', '上海市', '重庆市')
					)b
					where rnk = 1 and public_year = '2023'
			)c
			where deleted = '0' 
			group by public_year, national, province, city
			order by public_year
		)d
		left join
		(
		   select NATIONAL, province, city,
		   min(case when norm_type = '地方公共财政收入' and public_year = '2020'then cast( norm_value as float ) else null end) as '地方公共财政收入_2020',
		   min(case when norm_type = '地方公共财政收入' and public_year = '2021'then cast( norm_value as float ) else null end) as '地方公共财政收入_2021',
		   min(case when norm_type = '地方公共财政收入' and public_year = '2022'then cast( norm_value as float ) else null end) as '地方公共财政收入_2022',
		   min(case when norm_type = '地方公共财政收入' and public_year = '2023'then cast( norm_value as float ) else null end) as '地方公共财政收入_2023',  -- 有335个城市有数据
		   min(case when norm_type = '地区生产总值' and public_year = '2020' then cast( norm_value as float ) else null end) as '地区生产总值_2020',
		   min(case when norm_type = '地区生产总值' and public_year = '2021' then cast( norm_value as float ) else null end) as '地区生产总值_2021',
		   min(case when norm_type = '地区生产总值' and public_year = '2022' then cast( norm_value as float ) else null end) as '地区生产总值_2022',
		   min(case when norm_type = '地区生产总值' and public_year = '2023' then cast( norm_value as float ) else null end) as '地区生产总值_2023',   -- 有361个城市有数据
		   min(case when norm_type = '地区常住人口' and public_year = '2020' then cast( norm_value as float ) else null end) as '地区常住人口_2020',
		   min(case when norm_type = '地区常住人口' and public_year = '2021' then cast( norm_value as float ) else null end) as '地区常住人口_2021',
		   min(case when norm_type = '地区常住人口' and public_year = '2022' then cast( norm_value as float ) else null end) as '地区常住人口_2022',
		   min(case when norm_type = '地区常住人口' and public_year = '2023' then cast( norm_value as float ) else null end) as '地区常住人口_2023'     -- 有361个城市有数据
		   from
			(
				select * from
					(
						select *, ROW_NUMBER() over(PARTITION by public_year, national, province, city, norm_type order by update_time desc) as 'rnk'
						from jdbc_mysql_seed.dws_seed_storage_db.dws_area_norm
						where deleted = '0' and up_state != '3'
					)e
				where rnk = 1
			)f
			where deleted = '0' 
			group by national, province, city
		)h
		on d.national = h.national and d.province = h.province and d.city = h.city
	)i on a.province = i.province
	where reg_code = 'C03'
	Union
	select * from jdbc_mysql_seed.dws_seed_storage_db.dws_seed_storage_basic a
	left join 
	(
		select d.*, h.地方公共财政收入_2020, h.地方公共财政收入_2021, h.地方公共财政收入_2022, h.地方公共财政收入_2023,
			        h.地区生产总值_2020, h.地区生产总值_2021, h.地区生产总值_2022, h.地区生产总值_2023,
			        h.地区常住人口_2020, h.地区常住人口_2021, h.地区常住人口_2022, h.地区常住人口_2023
		FROM 
		(
			select public_year, national, province, city,
			-- min(case when norm_type = '地区内城投有息负债' then cast( norm_value as float ) else null end) as '地区内城投有息负债',   在平台得分指标里面
			min(case when norm_type = '地方公共财政收入' then cast( norm_value as float ) else null end) as '地方公共财政收入',
			min(case when norm_type = '地区债务率' then cast( norm_value as float ) else null end) as '地区债务率',
			min(case when norm_type = '地区偿债率' then cast( norm_value as float ) else null end) as '地区偿债率',
			min(case when norm_type = '地区财政自给率' then cast( norm_value as float ) else null end) as '地区财政自给率',
			min(case when norm_type = '地区财政依存度' then cast( norm_value as float ) else null end) as '地区财政依存度',
			min(case when norm_type = '地区财政收入质量' then cast( norm_value as float ) else null end) as '地区财政收入质量',
			min(case when norm_type = '地区生产总值' then cast( norm_value as float ) else null end) as '地区生产总值',
			min(case when norm_type = '人均地区生产总值' then cast( norm_value as float ) else null end) as '人均地区生产总值',
			min(case when norm_type = '地区第三产业占比' then cast( norm_value as float ) else null end) as '地区第三产业占比',
			(select cast((select norm_value from jdbc_mysql_seed.dws_seed_storage_db.dws_area_norm where norm_type = '人均国内生产总值' and public_year = '2023') as float)) as '人均国内生产总值',
			min(case when norm_type = '地区常住人口' then cast( norm_value as float ) else null end) as '地区常住人口'
			from
			(
				select * from
					(
						select *, ROW_NUMBER() over(PARTITION by public_year, national, province, city, norm_type order by update_time desc) as 'rnk'
						from jdbc_mysql_seed.dws_seed_storage_db.dws_area_norm
						where deleted = '0' and up_state != '3' and city != ''
					)b
					where rnk = 1 and public_year = '2023'
			)c
			where deleted = '0' 
			group by public_year, national, province, city
			order by public_year
		)d
		left join
		(
		   select NATIONAL, province, city,
		   min(case when norm_type = '地方公共财政收入' and public_year = '2020'then cast( norm_value as float ) else null end) as '地方公共财政收入_2020',
		   min(case when norm_type = '地方公共财政收入' and public_year = '2021'then cast( norm_value as float ) else null end) as '地方公共财政收入_2021',
		   min(case when norm_type = '地方公共财政收入' and public_year = '2022'then cast( norm_value as float ) else null end) as '地方公共财政收入_2022',
		   min(case when norm_type = '地方公共财政收入' and public_year = '2023'then cast( norm_value as float ) else null end) as '地方公共财政收入_2023',  -- 有335个城市有数据
		   min(case when norm_type = '地区生产总值' and public_year = '2020' then cast( norm_value as float ) else null end) as '地区生产总值_2020',
		   min(case when norm_type = '地区生产总值' and public_year = '2021' then cast( norm_value as float ) else null end) as '地区生产总值_2021',
		   min(case when norm_type = '地区生产总值' and public_year = '2022' then cast( norm_value as float ) else null end) as '地区生产总值_2022',
		   min(case when norm_type = '地区生产总值' and public_year = '2023' then cast( norm_value as float ) else null end) as '地区生产总值_2023',   -- 有361个城市有数据
		   min(case when norm_type = '地区常住人口' and public_year = '2020' then cast( norm_value as float ) else null end) as '地区常住人口_2020',
		   min(case when norm_type = '地区常住人口' and public_year = '2021' then cast( norm_value as float ) else null end) as '地区常住人口_2021',
		   min(case when norm_type = '地区常住人口' and public_year = '2022' then cast( norm_value as float ) else null end) as '地区常住人口_2022',
		   min(case when norm_type = '地区常住人口' and public_year = '2023' then cast( norm_value as float ) else null end) as '地区常住人口_2023'     -- 有361个城市有数据
		   from
			(
				select * from
					(
						select *, ROW_NUMBER() over(PARTITION by public_year, national, province, city, norm_type order by update_time desc) as 'rnk'
						from jdbc_mysql_seed.dws_seed_storage_db.dws_area_norm
						where deleted = '0' and up_state != '3'
					)e
				where rnk = 1
			)f
			where deleted = '0' 
			group by national, province, city
		)h
		on d.national = h.national and d.province = h.province and d.city = h.city
	)i on a.city = i.city
	where reg_code = 'C04'
) A
left JOIN 
(
   select *,
   case when reg_status like '%注吊销%' or (reg_status like '%注销%' and reg_status not like '%未注销%' and reg_status not like '%未办理注销%') then '注销'
        when reg_status like '%吊销%' then '吊销'
        when reg_status like '%撤销%' or reg_status like '%撤消' then '撤销'
        when reg_status like '%停业%' then '停业'
        when reg_status like '%歇业%' then '歇业'
        when reg_status like '%清算%' then '清算'
        when reg_status like '%除名%' then '除名'
        when reg_status like '%责令关闭%' then '责令关闭'
        when reg_status like '%迁入%' then '迁入'
        when reg_status like '%迁出%' or reg_status like '%迁出外省%' or reg_status like '%迁出市外%' then '迁出'
        when reg_status like '%存续%' or reg_status like '%在营%' or reg_status like '%在业%' or reg_status like '%开业%' 
        	or (reg_status like '%正常%' or reg_status not like '%非正常%') or reg_status like '%存在%' then '在续'
        else reg_status end as 'status'
	from 
	(
		select * from
			(
				select *, ROW_NUMBER() over(PARTITION by ccx_code order by update_time desc) as 'rnk'
				from ccx_ent_info_db.basic
				where deleted = '0' 
			)a
		where rnk = 1 and up_state != '3'
	) b
	where deleted = '0'
) B
on A.ccx_code = B.ccx_code
'''
dfrz_qy_index = pd.read_sql(sql_dfrz_qy_index,con)

In [131]:
dfrz_qy_index.shape

(15206, 30)

In [132]:
dfrz_qy_index.head()

,statis_month,ccx_code,ent_name,uscc,reg_code,地方公共财政收入,地区债务率,地区偿债率,地区财政自给率,地区财政依存度,地区财政收入质量,地区生产总值,人均地区生产总值,地区第三产业占比,人均国内生产总值,地区常住人口,地方公共财政收入_2020,地方公共财政收入_2021,地方公共财政收入_2022,地方公共财政收入_2023,地区生产总值_2020,地区生产总值_2021,地区生产总值_2022,地区生产总值_2023,地区常住人口_2020,地区常住人口_2021,地区常住人口_2022,地区常住人口_2023,create_time,status
0,2024-07,CCX239160344411ulv,九江市国诚资产运营有限公司,91360400MA394PBW2Q,C04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89358.0,NaN,286.4400,292.2,301.00,NaN,3240.5027,3735.6800,4026.5967,NaN,460.0276,456.07,455.7700,NaN,2024-07-23 09:32:24,在续
1,2024-07,CCX241181997174HQh,洛阳通和公路建设有限公司,91410323MA3X5Y3P6P,C04,404.3,NaN,NaN,0.606147,0.073756,0.656938,5481.60,77434.0,57.100000,89358.0,707.90,383.9000,397.9,398.23,404.3,5128.3640,5447.1230,5675.1895,5481.60,705.9100,706.90,707.9000,707.90,2024-07-23 09:32:24,在续
2,2024-07,CCX242348011986udK,嘉兴市铁路与轨道交通投资集团有限责任公司,91330401MA2BB00R15,C04,NaN,NaN,NaN,NaN,NaN,NaN,7062.45,126851.0,44.523502,89358.0,558.40,NaN,NaN,NaN,NaN,5509.5200,6355.2800,6739.0000,7062.45,541.1000,551.60,555.1000,558.40,2024-07-23 09:32:24,在续
3,2024-07,CCX248656252618GQl,山西兰花集团晋城滨河家园物业服务有限公司,911405225684506373,C04,277.2,NaN,NaN,0.642857,0.118802,0.504690,2333.30,107003.0,32.000000,89358.0,217.18,145.2546,201.3,283.00,277.2,1425.7000,1912.4163,2305.4353,2333.30,219.4360,218.86,218.9267,217.18,2024-07-23 09:32:24,在续
4,2024-07,CCX251604847878dMv,建平磷铁矿业有限公司,9121132275279689XN,C04,82.7,NaN,NaN,0.259899,0.079237,NaN,1043.70,37368.0,48.300000,89358.0,278.80,77.8000,84.3,79.40,82.7,875.6000,944.7806,995.0000,1043.70,287.0000,284.00,279.8000,278.80,2024-07-23 09:32:24,在续


### 4.2.2 平台得分

In [251]:
sql_dfrz_pt_index = '''
select C.*, D.status from
(
	select CONCAT(YEAR(CURDATE()), '-', LPAD(MONTH(CURDATE()), 2, '0')) as 'statis_month', A.ccx_code, A.ent_name, A.uscc , A.reg_code, A.总资产收益率, A.营业收入, 
		A.净资产, A.总资产, A.营业外收入, A.利润总额, A.资本公积, A.实收资本, A.货币资金, A.流动负债, A.EBITDA利息保障倍数, A.对外担保, A.有息负债, A.资产负债率, A.短期有息负债,
		B.地区内城投有息负债, B.最新主体评级, B.平台行政级别, B.发债规模, B.第一大股东持股比例
	from
	(
		select a.ccx_code, a.ent_name, a.uscc , a.reg_code, d.总资产收益率, d.营业收入, d.净资产, d.总资产, d.营业外收入, d.利润总额, 
		d.资本公积, d.实收资本, d.货币资金, d.流动负债, d.EBITDA利息保障倍数, d.对外担保, d.有息负债, d.资产负债率, d.短期有息负债
		from jdbc_mysql_seed.dws_seed_storage_db.dws_seed_storage_basic a
		left join 
		(
			select year, ccx_code, ent_name,
				min(case when index_name = '总资产收益率' then cast( index_value as float ) else null end) as '总资产收益率',
				min(case when index_name= '营业收入' then cast( index_value as float ) else null end) as '营业收入',
				min(case when index_name = '净资产' then cast( index_value as float ) else null end) as '净资产',
				min(case when index_name = '总资产' then cast( index_value as float ) else null end) as '总资产',
				min(case when index_name = '营业外收入' then cast( index_value as float ) else null end) as '营业外收入',
				min(case when index_name = '利润总额' then cast( index_value as float ) else null end) as '利润总额',
				min(case when index_name = '资本公积' then cast( index_value as float ) else null end) as '资本公积',
				min(case when index_name = '实收资本' then cast( index_value as float ) else null end) as '实收资本',
				min(case when index_name = '货币资金' then cast( index_value as float ) else null end) as '货币资金',
				min(case when index_name = '流动负债' then cast( index_value as float ) else null end) as '流动负债',
				min(case when index_name = 'EBITDA利息保障倍数' then cast( index_value as float ) else null end) as 'EBITDA利息保障倍数',
				min(case when index_name = '对外担保' then cast( index_value as float ) else null end) as '对外担保',
				min(case when index_name = '有息负债' then cast( index_value as float ) else null end) as '有息负债',
				min(case when index_name = '资产负债率' then cast( index_value as float ) else null end) as '资产负债率',
				min(case when index_name = '短期有息负债' then cast( index_value as float ) else null end) as '短期有息负债'
				from 
				(
					select * from
					(
						select *, ROW_NUMBER() over(PARTITION by year, ccx_code, ent_name, index_name order by update_time desc) as 'rnk'
						from jdbc_mysql_seed.dws_seed_storage_db.dws_ent_finance
						where deleted = '0' and up_state != '3'
					)b
					where rnk = 1
				)c
				group by year, ccx_code, ent_name
				order by year
		)d on a.ccx_code = d.ccx_code
		where reg_code in ('C03', 'C04')
	) A
	left join
	(
		select e.ccx_code, e.ent_name, e.uscc , e.reg_code, h.地区内城投有息负债, h.最新主体评级, h.平台行政级别, h.发债规模, h.第一大股东持股比例
		from jdbc_mysql_seed.dws_seed_storage_db.dws_seed_storage_basic e
		left join 
		(
			select ccx_code, ent_name, cast(debt as float) as '地区内城投有息负债', last_subject_rate as '最新主体评级', 
				  district_level as '平台行政级别', cast(bond_scale as float) as '发债规模', cast(largest_shareholder_rate as float) as '第一大股东持股比例'
			from 
			(
				select * from
				(
					select *, ROW_NUMBER() over(PARTITION by ccx_code,  ent_name order by update_time desc) as 'rnk'
					from jdbc_mysql_seed.dws_seed_storage_db.dws_urban_investment_bonds
					where deleted = '0' and up_state != '3'
				)f
				where rnk = 1
			)g
		)h on e.ccx_code = h.ccx_code
		where reg_code in ('C03', 'C04')
	) B
	on A.ccx_code = B.ccx_code and A.ent_name = B.ent_name and A.uscc = B.uscc and A.reg_code = B.reg_code
) C
left join
(
   select *,
   case when reg_status like '%注吊销%' or (reg_status like '%注销%' and reg_status not like '%未注销%' and reg_status not like '%未办理注销%') then '注销'
        when reg_status like '%吊销%' then '吊销'
        when reg_status like '%撤销%' or reg_status like '%撤消' then '撤销'
        when reg_status like '%停业%' then '停业'
        when reg_status like '%歇业%' then '歇业'
        when reg_status like '%清算%' then '清算'
        when reg_status like '%除名%' then '除名'
        when reg_status like '%责令关闭%' then '责令关闭'
        when reg_status like '%迁入%' then '迁入'
        when reg_status like '%迁出%' or reg_status like '%迁出外省%' or reg_status like '%迁出市外%' then '迁出'
        when reg_status like '%存续%' or reg_status like '%在营%' or reg_status like '%在业%' or reg_status like '%开业%' 
        	or (reg_status like '%正常%' or reg_status not like '%非正常%') or reg_status like '%存在%' then '在续'
        else reg_status end as 'status'
	from 
	(
		select * from
			(
				select *, ROW_NUMBER() over(PARTITION by ccx_code order by update_time desc) as 'rnk'
				from ccx_ent_info_db.basic
				where deleted = '0' 
			)f
		where rnk = 1 and up_state != '3'
	) g
	where deleted = '0'
) D
on C.ccx_code = D.ccx_code
'''
dfrz_pt_index = pd.read_sql(sql_dfrz_pt_index,con)

In [184]:
dfrz_pt_index.shape

(15206, 25)

In [185]:
dfrz_pt_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15206 entries, 0 to 15205
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   statis_month  15206 non-null  object 
 1   ccx_code      15154 non-null  object 
 2   ent_name      15206 non-null  object 
 3   uscc          15056 non-null  object 
 4   reg_code      15206 non-null  object 
 5   总资产收益率        1072 non-null   float64
 6   营业收入          1071 non-null   float64
 7   净资产           1072 non-null   float64
 8   总资产           1072 non-null   float64
 9   营业外收入         1057 non-null   float64
 10  利润总额          1070 non-null   float64
 11  资本公积          1058 non-null   float64
 12  货币资本          0 non-null      object 
 13  流动负债          992 non-null    float64
 14  EBITDA利息保障倍数  1072 non-null   float64
 15  对外担保          10 non-null     float64
 16  有息负债          1072 non-null   float64
 17  资产负债率         1072 non-null   float64
 18  短期有息负债        1072 non-nul

In [118]:
dfrz_pt_index.head()

,statis_month,ccx_code,ent_name,uscc,reg_code,总资产收益率,营业收入,净资产,总资产,营业外收入,利润总额,资本公积,货币资本,流动负债,EBITDA利息保障倍数,对外担保,有息负债,资产负债率,短期有息负债,地区内城投有息负债,最新主体评级,平台行政级别,发债规模,第一大股东持股比例,status
0,2024-07,CCX238904488828xeG,哈尔滨市报达印刷物资运输有限公司,91230199731347999D,C04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,在续
1,2024-07,CCX239042900452UFC,湖南娄底农村商业银行股份有限公司,914313001874408810,C04,0.005881,293259550.0,748974000.0,1.407339e+10,403064.3,93259550.0,108676230.0,None,NaN,-999999.0,NaN,-999999.0,0.946781,-999999.0,None,None,None,None,None,在续
2,2024-07,CCX249109237043BZU,中嘉数字科技（浙江）有限公司,91330109MA2H2C1065,C04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,在续
3,2024-07,CCX251806173144MFp,秦皇岛秦旅科技发展有限公司,91130304MA09THY65E,C04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,在续
4,2024-07,CCX252418543313TtL,平顶山市高速公路建设有限责任公司,91410400721876424C,C04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,在续


In [95]:
sql_dfrz_qy_recentyear_index = '''
select d.NATIONAL, d.province, d.city, 
       CONCAT(ROUND(地区近三年财政收入平均增长率, 2), '%'),
       CONCAT(ROUND(地区近三年GDP平均增速, 2), '%'), 
       CONCAT(ROUND(地区近三年常住人口变化率, 2), '%')
from
(
	select NATIONAL, province, city, cast(POWER((地方公共财政收入_2023/地方公共财政收入_2020), 1/3) - 1 as float) * 100 as '地区近三年财政收入平均增长率', 
	       cast(POWER((地区生产总值_2023/地区生产总值_2020), 1/3) - 1 as float) * 100 as '地区近三年GDP平均增速',
	       cast(POWER((地区常住人口_2023/地区常住人口_2020), 1/3) - 1 as float) * 100 as'地区近三年常住人口变化率'
	from
	(
	select NATIONAL, province, city,
		   min(case when norm_type = '地方公共财政收入' and public_year = '2019'then cast( norm_value as float ) else null end) as '地方公共财政收入_2019',
		   min(case when norm_type = '地方公共财政收入' and public_year = '2020'then cast( norm_value as float ) else null end) as '地方公共财政收入_2020',
		   min(case when norm_type = '地方公共财政收入' and public_year = '2021'then cast( norm_value as float ) else null end) as '地方公共财政收入_2021',
		   min(case when norm_type = '地方公共财政收入' and public_year = '2022'then cast( norm_value as float ) else null end) as '地方公共财政收入_2022',
		   min(case when norm_type = '地方公共财政收入' and public_year = '2023'then cast( norm_value as float ) else null end) as '地方公共财政收入_2023',
		   min(case when norm_type = '地区生产总值' and public_year = '2019' then cast( norm_value as float ) else null end) as '地区生产总值_2019',
		   min(case when norm_type = '地区生产总值' and public_year = '2020' then cast( norm_value as float ) else null end) as '地区生产总值_2020',
		   min(case when norm_type = '地区生产总值' and public_year = '2021' then cast( norm_value as float ) else null end) as '地区生产总值_2021',
		   min(case when norm_type = '地区生产总值' and public_year = '2022' then cast( norm_value as float ) else null end) as '地区生产总值_2022',
		   min(case when norm_type = '地区生产总值' and public_year = '2023' then cast( norm_value as float ) else null end) as '地区生产总值_2023',
		   min(case when norm_type = '地区常住人口' and public_year = '2019' then cast( norm_value as float ) else null end) as '地区常住人口_2019',
		   min(case when norm_type = '地区常住人口' and public_year = '2020' then cast( norm_value as float ) else null end) as '地区常住人口_2020',
		   min(case when norm_type = '地区常住人口' and public_year = '2021' then cast( norm_value as float ) else null end) as '地区常住人口_2021',
		   min(case when norm_type = '地区常住人口' and public_year = '2022' then cast( norm_value as float ) else null end) as '地区常住人口_2022',
		   min(case when norm_type = '地区常住人口' and public_year = '2023' then cast( norm_value as float ) else null end) as '地区常住人口_2023'
		   from
			(
				select * from
					(
						select *, ROW_NUMBER() over(PARTITION by public_year, national, province, city, norm_type order by update_time desc) as 'rnk'
						from jdbc_mysql_seed.dws_seed_storage_db.dws_area_norm
						where deleted = '0' and up_state != '3'
					)a
				where rnk = 1
			)b
		where deleted = '0' 
		group by national, province, city
	) c
)d
'''
dfrz_qy_recentyear_index = pd.read_sql(sql_dfrz_qy_recentyear_index,con)

In [96]:
dfrz_qy_recentyear_index.shape

(364, 6)

In [97]:
dfrz_qy_recentyear_index.head()

,NATIONAL,province,city,"concat(round(地区近三年财政收入平均增长率, 2), '%')","concat(round(地区近三年GDP平均增速, 2), '%')","concat(round(地区近三年常住人口变化率, 2), '%')"
0,中国,内蒙古自治区,兴安盟,-6.26%,8.65%,-0.67%
1,中国,四川省,甘孜藏族自治州,None,7.73%,None
2,中国,四川省,达州市,17.6%,7.85%,-0.39%
3,中国,山东省,聊城市,6.59%,8.1%,-0.52%
4,中国,山西省,太原市,None,10.3%,None


In [105]:
len(dfrz_qy_recentyear_index[dfrz_qy_recentyear_index['city'] == ''])

26

In [91]:
sql_dfrz_pt_index = '''
select C.*, D.status from
(
	select CONCAT(YEAR(CURDATE()), '-', LPAD(MONTH(CURDATE()), 2, '0')) as 'statis_month', A.ccx_code, A.ent_name, A.uscc , A.reg_code, A.总资产收益率, A.营业收入, 
		A.净资产, A.总资产, A.营业外收入, A.利润总额, A.资本公积, A.货币资本, A.流动负债, A.EBITDA利息保障倍数, A.对外担保, A.有息负债, A.资产负债率, A.短期有息负债,
		B.地区内城投有息负债, B.最新主体评级, B.平台行政级别, B.发债规模, B.第一大股东持股比例
	from
	(
		select a.ccx_code, a.ent_name, a.uscc , a.reg_code, d.总资产收益率, d.营业收入, d.净资产, d.总资产, d.营业外收入, d.利润总额, 
		d.资本公积, d.货币资本, d.流动负债, d.EBITDA利息保障倍数, d.对外担保, d.有息负债, d.资产负债率, d.短期有息负债
		from jdbc_mysql_seed.dws_seed_storage_db.dws_seed_storage_basic a
		left join 
		(
			select year, ccx_code, ent_name,
				min(case when index_name = '总资产收益率' then cast( index_value as float ) else null end) as '总资产收益率',
				min(case when index_name= '营业收入' then cast( index_value as float ) else null end) as '营业收入',
				min(case when index_name = '净资产' then cast( index_value as float ) else null end) as '净资产',
				min(case when index_name = '总资产' then cast( index_value as float ) else null end) as '总资产',
				min(case when index_name = '营业外收入' then cast( index_value as float ) else null end) as '营业外收入',
				min(case when index_name = '利润总额' then cast( index_value as float ) else null end) as '利润总额',
				min(case when index_name = '资本公积' then cast( index_value as float ) else null end) as '资本公积',
				min(case when index_name = '货币资本' then cast( index_value as float ) else null end) as '货币资本',
				min(case when index_name = '流动负债' then cast( index_value as float ) else null end) as '流动负债',
				min(case when index_name = 'EBITDA利息保障倍数' then cast( index_value as float ) else null end) as 'EBITDA利息保障倍数',
				min(case when index_name = '对外担保' then cast( index_value as float ) else null end) as '对外担保',
				min(case when index_name = '有息负债' then cast( index_value as float ) else null end) as '有息负债',
				min(case when index_name = '资产负债率' then cast( index_value as float ) else null end) as '资产负债率',
				min(case when index_name = '短期有息负债' then cast( index_value as float ) else null end) as '短期有息负债'
				from 
				(
					select * from
					(
						select *, ROW_NUMBER() over(PARTITION by year, ccx_code, ent_name, index_name order by update_time desc) as 'rnk'
						from jdbc_mysql_seed.dws_seed_storage_db.dws_ent_finance
						where deleted = '0' and up_state != '3'
					)b
					where rnk = 1
				)c
				group by year, ccx_code, ent_name
				order by year
		)d on a.ccx_code = d.ccx_code
		where reg_code in ('C03', 'C04')
	) A
	left join
	(
		select e.ccx_code, e.ent_name, e.uscc , e.reg_code, h.地区内城投有息负债, h.最新主体评级, h.平台行政级别, h.发债规模, h.第一大股东持股比例
		from jdbc_mysql_seed.dws_seed_storage_db.dws_seed_storage_basic e
		left join 
		(
			select ccx_code, ent_name, debt as '地区内城投有息负债', last_subject_rate as '最新主体评级', district_level as '平台行政级别', bond_scale as '发债规模', largest_shareholder_rate as '第一大股东持股比例'
			from 
			(
				select * from
				(
					select *, ROW_NUMBER() over(PARTITION by ccx_code,  ent_name order by update_time desc) as 'rnk'
					from jdbc_mysql_seed.dws_seed_storage_db.dws_urban_investment_bonds
					where deleted = '0' and up_state != '3'
				)f
				where rnk = 1
			)g
		)h on e.ccx_code = h.ccx_code
		where reg_code in ('C03', 'C04')
	) B
	on A.ccx_code = B.ccx_code and A.ent_name = B.ent_name and A.uscc = B.uscc and A.reg_code = B.reg_code
) C
left join
(
   select *,
   case when reg_status like '%注吊销%' or (reg_status like '%注销%' and reg_status not like '%未注销%' and reg_status not like '%未办理注销%') then '注销'
        when reg_status like '%吊销%' then '吊销'
        when reg_status like '%撤销%' or reg_status like '%撤消' then '撤销'
        when reg_status like '%停业%' then '停业'
        when reg_status like '%歇业%' then '歇业'
        when reg_status like '%清算%' then '清算'
        when reg_status like '%除名%' then '除名'
        when reg_status like '%责令关闭%' then '责令关闭'
        when reg_status like '%迁入%' then '迁入'
        when reg_status like '%迁出%' or reg_status like '%迁出外省%' or reg_status like '%迁出市外%' then '迁出'
        when reg_status like '%存续%' or reg_status like '%在营%' or reg_status like '%在业%' or reg_status like '%开业%' 
        	or (reg_status like '%正常%' or reg_status not like '%非正常%') or reg_status like '%存在%' then '在续'
        else reg_status end as 'status'
	from 
	(
		select * from
			(
				select *, ROW_NUMBER() over(PARTITION by ccx_code order by update_time desc) as 'rnk'
				from ccx_ent_info_db.basic
				where deleted = '0' 
			)f
		where rnk = 1 and up_state != '3'
	) g
	where deleted = '0'
) D
on C.ccx_code = D.ccx_code
'''
dfrz_pt_index = pd.read_sql(sql_dqjj_index,con)

In [94]:
dfrz_pt_index.shape

(15206, 17)

In [92]:
dfrz_pt_index.head()

,statis_month,ccx_code,ent_name,uscc,reg_code,地区生产总值,人均地区生产总值,地区经济发展趋势,地方公共财政收入,地区财政收入质量,地区财政依存度,地区土地财政依赖度,地区财政自给率,地区负债率,地区债务率,create_time,status
0,2024-07,CCX15434088897054Q,潮州市龙燕矿业有限公司,91445102MACKPK0F08,C04,1356.59,52665.0,103.0,59.65,NaN,0.043971,NaN,0.295604,NaN,NaN,2024-07-22 11:25:41,在续
1,2024-07,CCX242251544419ZyP,泉州国泰房地产开发有限公司,91350500746393437M,C04,12172.33,137060.0,104.8,580.79,NaN,0.047714,NaN,0.683065,0.192448,NaN,2024-07-22 11:25:41,在续
2,2024-07,CCX248979211543GpR,湛江市交通投资集团有限公司,91440800737583861G,C04,3793.59,53757.0,103.0,155.61,NaN,0.041019,NaN,0.286401,0.293674,NaN,2024-07-22 11:25:41,在续
3,2024-07,CCX249830655950lFX,天长市农业发展有限公司,91341181MA2TQCYR97,C04,3782.01,93325.0,106.4,298.20,NaN,0.078847,NaN,0.558636,NaN,NaN,2024-07-22 11:25:41,在续
4,2024-07,CCX254557637453lli,乐山国有资本运营有限公司,91511100MA632LQN20,C04,2447.53,77699.0,106.5,159.60,0.541792,0.065209,NaN,0.442694,NaN,NaN,2024-07-22 11:25:41,在续


## 4.3 地区融资评分卡指标加工

In [ ]:
'''
地方融资评分卡区域得分 = 
[
'城区内城投有息负债/地方公共财政收入',
'地区债务率',
'地区偿债率',
'地区财政自给率',
'地方公共财政收入',
'地区近三年财政收入平均增长率',
'地区财政依存度',
'地区财政收入质量',
'地区生产总值',
'人均地区生产总值',
'地区近三年GDP平均增速',
'地区第三产业占比',
'人均地区生产总值/人均国内生产总值',
'地区近三年常住人口变化率'
]
地方融资评分卡平台得分 = 
[
'总资产收益率',
'营业收入',
'净资产',
'总资产',
'最新主体评级',
'行政级别',
'营业外收入/利润总额',
'营业外收入/营业收入',
'发债规模/净资产',
'第一大股东持股比例',
'(实收资本+资本公积)/净资产'
'货币资金/流动负债',
'EBITDA利息保障倍数',
'对外担保/净资产',
'有息负债/负债合计',
'资产负债率',
'短期有息负债/有息负债'
]
'''

### 4.3.1 指标合并

In [252]:
## 将4.2三种指标进行聚合
# 区域得分指标与平台得分指标进行合并
dfrz_index = pd.merge(dfrz_qy_index, dfrz_pt_index, on = ['ent_name', 'reg_code'])

In [253]:
dfrz_index.head()

,statis_month_x,ccx_code_x,ent_name,uscc_x,reg_code,地方公共财政收入,地区债务率,地区偿债率,地区财政自给率,地区财政依存度,地区财政收入质量,地区生产总值,人均地区生产总值,地区第三产业占比,人均国内生产总值,地区常住人口,地方公共财政收入_2020,地方公共财政收入_2021,地方公共财政收入_2022,地方公共财政收入_2023,地区生产总值_2020,地区生产总值_2021,地区生产总值_2022,地区生产总值_2023,地区常住人口_2020,地区常住人口_2021,地区常住人口_2022,地区常住人口_2023,create_time,status_x,statis_month_y,ccx_code_y,uscc_y,总资产收益率,营业收入,净资产,总资产,营业外收入,利润总额,资本公积,实收资本,货币资金,流动负债,EBITDA利息保障倍数,对外担保,有息负债,资产负债率,短期有息负债,地区内城投有息负债,最新主体评级,平台行政级别,发债规模,第一大股东持股比例,status_y
0,2024-07,CCX239160344411ulv,九江市国诚资产运营有限公司,91360400MA394PBW2Q,C04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89358.0,NaN,286.4400,292.2,301.00,NaN,3240.5027,3735.6800,4026.5967,NaN,460.0276,456.07,455.7700,NaN,2024-07-23 09:32:24,在续,2024-07,CCX239160344411ulv,91360400MA394PBW2Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,在续
1,2024-07,CCX241181997174HQh,洛阳通和公路建设有限公司,91410323MA3X5Y3P6P,C04,404.3,NaN,NaN,0.606147,0.073756,0.656938,5481.60,77434.0,57.100000,89358.0,707.90,383.9000,397.9,398.23,404.3,5128.3640,5447.1230,5675.1895,5481.60,705.9100,706.90,707.9000,707.90,2024-07-23 09:32:24,在续,2024-07,CCX241181997174HQh,91410323MA3X5Y3P6P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,在续
2,2024-07,CCX242348011986udK,嘉兴市铁路与轨道交通投资集团有限责任公司,91330401MA2BB00R15,C04,NaN,NaN,NaN,NaN,NaN,NaN,7062.45,126851.0,44.523502,89358.0,558.40,NaN,NaN,NaN,NaN,5509.5200,6355.2800,6739.0000,7062.45,541.1000,551.60,555.1000,558.40,2024-07-23 09:32:24,在续,2024-07,CCX242348011986udK,91330401MA2BB00R15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,在续
3,2024-07,CCX248656252618GQl,山西兰花集团晋城滨河家园物业服务有限公司,911405225684506373,C04,277.2,NaN,NaN,0.642857,0.118802,0.504690,2333.30,107003.0,32.000000,89358.0,217.18,145.2546,201.3,283.00,277.2,1425.7000,1912.4163,2305.4353,2333.30,219.4360,218.86,218.9267,217.18,2024-07-23 09:32:24,在续,2024-07,CCX248656252618GQl,911405225684506373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,在续
4,2024-07,CCX251604847878dMv,建平磷铁矿业有限公司,9121132275279689XN,C04,82.7,NaN,NaN,0.259899,0.079237,NaN,1043.70,37368.0,48.300000,89358.0,278.80,77.8000,84.3,79.40,82.7,875.6000,944.7806,995.0000,1043.70,287.0000,284.00,279.8000,278.80,2024-07-23 09:32:24,在续,2024-07,CCX251604847878dMv,9121132275279689XN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,在续


In [201]:
dfrz_index.shape

(15208, 53)

In [202]:
dfrz_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15208 entries, 0 to 15207
Data columns (total 53 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   statis_month_x  15208 non-null  object        
 1   ccx_code_x      15154 non-null  object        
 2   ent_name        15208 non-null  object        
 3   uscc_x          15056 non-null  object        
 4   reg_code        15208 non-null  object        
 5   地方公共财政收入        13136 non-null  float64       
 6   地区债务率           2736 non-null   float64       
 7   地区偿债率           523 non-null    float64       
 8   地区财政自给率         12921 non-null  float64       
 9   地区财政依存度         12901 non-null  float64       
 10  地区财政收入质量        3285 non-null   float64       
 11  地区生产总值          14669 non-null  float64       
 12  人均地区生产总值        14386 non-null  float64       
 13  地区第三产业占比        11925 non-null  float64       
 14  人均国内生产总值        15133 non-null  float64       
 15  地区

In [254]:
# 新疆博州温泉县呼和托哈种蓄场  所有信息都是一致的，所以会重复匹配
dfrz_index[dfrz_index['ent_name'] == '新疆博州温泉县呼和托哈种蓄场']

,statis_month_x,ccx_code_x,ent_name,uscc_x,reg_code,地方公共财政收入,地区债务率,地区偿债率,地区财政自给率,地区财政依存度,地区财政收入质量,地区生产总值,人均地区生产总值,地区第三产业占比,人均国内生产总值,地区常住人口,地方公共财政收入_2020,地方公共财政收入_2021,地方公共财政收入_2022,地方公共财政收入_2023,地区生产总值_2020,地区生产总值_2021,地区生产总值_2022,地区生产总值_2023,地区常住人口_2020,地区常住人口_2021,地区常住人口_2022,地区常住人口_2023,create_time,status_x,statis_month_y,ccx_code_y,uscc_y,总资产收益率,营业收入,净资产,总资产,营业外收入,利润总额,资本公积,实收资本,货币资金,流动负债,EBITDA利息保障倍数,对外担保,有息负债,资产负债率,短期有息负债,地区内城投有息负债,最新主体评级,平台行政级别,发债规模,第一大股东持股比例,status_y
5154,2024-07,None,新疆博州温泉县呼和托哈种蓄场,None,C04,43.54,NaN,NaN,0.272841,0.08492,NaN,512.72,NaN,47.9,89358.0,NaN,28.8923,NaN,NaN,43.54,377.1371,448.0306,481.66,512.72,NaN,NaN,NaN,NaN,2024-07-23 09:32:24,None,2024-07,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None
5155,2024-07,None,新疆博州温泉县呼和托哈种蓄场,None,C04,43.54,NaN,NaN,0.272841,0.08492,NaN,512.72,NaN,47.9,89358.0,NaN,28.8923,NaN,NaN,43.54,377.1371,448.0306,481.66,512.72,NaN,NaN,NaN,NaN,2024-07-23 09:32:24,None,2024-07,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None
5156,2024-07,None,新疆博州温泉县呼和托哈种蓄场,None,C04,43.54,NaN,NaN,0.272841,0.08492,NaN,512.72,NaN,47.9,89358.0,NaN,28.8923,NaN,NaN,43.54,377.1371,448.0306,481.66,512.72,NaN,NaN,NaN,NaN,2024-07-23 09:32:24,None,2024-07,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None
5157,2024-07,None,新疆博州温泉县呼和托哈种蓄场,None,C04,43.54,NaN,NaN,0.272841,0.08492,NaN,512.72,NaN,47.9,89358.0,NaN,28.8923,NaN,NaN,43.54,377.1371,448.0306,481.66,512.72,NaN,NaN,NaN,NaN,2024-07-23 09:32:24,None,2024-07,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None


In [255]:
# 去除一样重复的值，全部字段都重复的样本
dfrz_qc_index = dfrz_index.drop_duplicates()

In [176]:
dfrz_qc_index.shape

(15205, 53)

In [256]:
# 去除重复的字段，并改名
dfrz_qc_index.drop(['statis_month_y', 'ccx_code_y', 'uscc_y', 'status_y'], axis = 1, inplace = True)

In [257]:
dfrz_qc_index.shape

(15205, 50)

In [258]:
dfrz_qc_index = dfrz_qc_index.rename(columns = {'statis_month_x': 'statis_month', 'ccx_code_x' : 'ccx_code', 'uscc_x': 'uscc', 'status_x': 'status'})

In [261]:
dfrz_qc_index.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15205 entries, 0 to 15207
Data columns (total 50 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   statis_month   15205 non-null  object        
 1   ccx_code       15154 non-null  object        
 2   ent_name       15205 non-null  object        
 3   uscc           15056 non-null  object        
 4   reg_code       15205 non-null  object        
 5   地方公共财政收入       13133 non-null  float64       
 6   地区债务率          2736 non-null   float64       
 7   地区偿债率          523 non-null    float64       
 8   地区财政自给率        12918 non-null  float64       
 9   地区财政依存度        12898 non-null  float64       
 10  地区财政收入质量       3285 non-null   float64       
 11  地区生产总值         14666 non-null  float64       
 12  人均地区生产总值       14386 non-null  float64       
 13  地区第三产业占比       11922 non-null  float64       
 14  人均国内生产总值       15130 non-null  float64       
 15  地区常住人口         11333 non

### 4.3.2 指标加工

In [260]:
dfrz_qc_index.columns

Index(['statis_month', 'ccx_code', 'ent_name', 'uscc', 'reg_code', '地方公共财政收入',
       '地区债务率', '地区偿债率', '地区财政自给率', '地区财政依存度', '地区财政收入质量', '地区生产总值',
       '人均地区生产总值', '地区第三产业占比', '人均国内生产总值', '地区常住人口', '地方公共财政收入_2020',
       '地方公共财政收入_2021', '地方公共财政收入_2022', '地方公共财政收入_2023', '地区生产总值_2020',
       '地区生产总值_2021', '地区生产总值_2022', '地区生产总值_2023', '地区常住人口_2020',
       '地区常住人口_2021', '地区常住人口_2022', '地区常住人口_2023', 'create_time', 'status',
       '总资产收益率', '营业收入', '净资产', '总资产', '营业外收入', '利润总额', '资本公积', '实收资本', '货币资金',
       '流动负债', 'EBITDA利息保障倍数', '对外担保', '有息负债', '资产负债率', '短期有息负债', '地区内城投有息负债',
       '最新主体评级', '平台行政级别', '发债规模', '第一大股东持股比例'],
      dtype='object')

In [ ]:
'''
地方融资评分卡区域得分 = 
[
'地区内城投有息负债/地方公共财政收入',
'地区债务率',
'地区偿债率',
'地区财政自给率',
'地方公共财政收入',
'地区近三年财政收入平均增长率',
'地区财政依存度',
'地区财政收入质量',
'地区生产总值',
'人均地区生产总值',
'地区近近三年GDP平均增速',
'地区第三产业占比',
'人均地区生产总值/人均国内生产总值',
'地区近三年常住人口变化率'
]
地方融资评分卡平台得分 = 
[
'总资产收益率',
'营业收入',
'净资产',
'总资产',
'最新主体评级',
'行政级别',
'营业外收入/利润总额',
'营业外收入/营业收入',
'发债规模/净资产',
'第一大股东持股比例',
'(实收资本+资本公积)/净资产'
'货币资金/流动负债',
'EBITDA利息保障倍数',
'对外担保/净资产',
'有息负债/负债合计',
'资产负债率',
'短期有息负债/有息负债'
]
'''

In [177]:
dfrz_qc_index.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15205 entries, 0 to 15207
Data columns (total 53 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   statis_month_x  15205 non-null  object        
 1   ccx_code_x      15154 non-null  object        
 2   ent_name        15205 non-null  object        
 3   uscc_x          15056 non-null  object        
 4   reg_code        15205 non-null  object        
 5   地方公共财政收入        13133 non-null  float64       
 6   地区债务率           2736 non-null   float64       
 7   地区偿债率           523 non-null    float64       
 8   地区财政自给率         12918 non-null  float64       
 9   地区财政依存度         12898 non-null  float64       
 10  地区财政收入质量        3285 non-null   float64       
 11  地区生产总值          14666 non-null  float64       
 12  人均地区生产总值        14386 non-null  float64       
 13  地区第三产业占比        11922 non-null  float64       
 14  人均国内生产总值        15130 non-null  float64       
 15  地区常住人口 

In [262]:
# 区域得分指标衍生
# 需要衍生的：
'''
'地区内城投有息负债/地方公共财政收入'
'人均地区生产总值/人均国内生产总值'
'地区近三年财政收入平均增长率'
'地区近近三年GDP平均增速'
'地区近三年常住人口变化率'
'''
dfrz_qc_index['城区内城投有息负债/地方公共财政收入'] = dfrz_qc_index[['地区内城投有息负债', '地方公共财政收入']].apply(lambda x: x[0]/x[1] if pd.notna(x[1]) else np.nan, axis=1)
dfrz_qc_index['人均地区生产总值/人均国内生产总值'] = dfrz_qc_index[['人均地区生产总值', '人均国内生产总值']].apply(lambda x: x[0]/x[1] if pd.notna(x[1]) else np.nan, axis=1)
dfrz_qc_index['地区近三年财政收入平均增长率'] = dfrz_qc_index[['地方公共财政收入_2020', '地方公共财政收入_2021', '地方公共财政收入_2022', '地方公共财政收入_2023']].apply(lambda x: pow((x[-1]-x[0]), 1/3) - 1 if pd.notna(x[1]) else np.nan, axis=1)
dfrz_qc_index['地区近近三年GDP平均增速'] = dfrz_qc_index[['地区生产总值_2020', '地区生产总值_2021', '地区生产总值_2022', '地区生产总值_2023']].apply(lambda x: pow((x[-1]-x[0]), 1/3) - 1 if pd.notna(x[1]) else np.nan, axis=1)
dfrz_qc_index['地区近三年常住人口变化率'] = dfrz_qc_index[['地区常住人口_2020', '地区常住人口_2021', '地区常住人口_2022', '地区常住人口_2023']].apply(lambda x: pow((x[-1]-x[0]), 1/3) - 1 if pd.notna(x[1]) else np.nan, axis=1)

In [264]:
# 平台得分指标衍生
# 需要衍生的
'''
'营业外收入/利润总额',
'营业外收入/营业收入',
'发债规模/净资产',
'(实收资本+资本公积)/净资产'
'货币资金/流动负债',
'对外担保/净资产',
'有息负债/负债合计',
'短期有息负债/有息负债'
'''
dfrz_qc_index['营业外收入/利润总额'] = dfrz_qc_index[['营业外收入', '利润总额']].apply(lambda x: x[0]/x[1] if pd.notna(x[1]) else np.nan, axis=1)
dfrz_qc_index['营业外收入/营业收入'] = dfrz_qc_index[['营业外收入', '营业收入']].apply(lambda x: x[0]/x[1] if pd.notna(x[1]) else np.nan, axis=1)
dfrz_qc_index['发债规模/净资产'] = dfrz_qc_index[['发债规模', '净资产']].apply(lambda x: x[0]/x[1] if pd.notna(x[1]) else np.nan, axis=1)
dfrz_qc_index['(实收资本+资本公积)/净资产'] = dfrz_qc_index[['实收资本', '资本公积', '净资产']].apply(lambda x: (x[0] + x[1])/x[2] if pd.notna(x[2]) else np.nan, axis=1)
dfrz_qc_index['货币资金/流动负债'] = dfrz_qc_index[['货币资金', '流动负债']].apply(lambda x: x[0]/x[1] if pd.notna(x[1]) else np.nan, axis=1)
dfrz_qc_index['对外担保/净资产'] = dfrz_qc_index[['对外担保', '净资产']].apply(lambda x: x[0]/x[1] if pd.notna(x[1]) else np.nan, axis=1)
dfrz_qc_index['短期有息负债/有息负债'] = dfrz_qc_index[['短期有息负债', '有息负债']].apply(lambda x: x[0]/x[1] if pd.notna(x[1]) else np.nan, axis=1)

In [ ]:
# 无负债合计  ----目前无法进行加工
dfrz_qc_index['有息负债/负债合计'] = dfrz_qc_index[['有息负债', '负债合计']].apply(lambda x: x[0]/x[1] if pd.notna(x[1]) else np.nan, axis=1)

In [265]:
dfrz_qc_index.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15205 entries, 0 to 15207
Data columns (total 62 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   statis_month        15205 non-null  object        
 1   ccx_code            15154 non-null  object        
 2   ent_name            15205 non-null  object        
 3   uscc                15056 non-null  object        
 4   reg_code            15205 non-null  object        
 5   地方公共财政收入            13133 non-null  float64       
 6   地区债务率               2736 non-null   float64       
 7   地区偿债率               523 non-null    float64       
 8   地区财政自给率             12918 non-null  float64       
 9   地区财政依存度             12898 non-null  float64       
 10  地区财政收入质量            3285 non-null   float64       
 11  地区生产总值              14666 non-null  float64       
 12  人均地区生产总值            14386 non-null  float64       
 13  地区第三产业占比            11922 non-null  float64       


In [238]:
(dfrz_qc_index['地区内城投有息负债'].notna() & dfrz_qc_index['地方公共财政收入'].notna()).sum()

504

In [266]:
(dfrz_qc_index['对外担保'].notna() & dfrz_qc_index['净资产'].notna()).sum()

3

## 4.4 缺失值异常值分析

In [270]:
dfrz_qc_index.describe().loc['min', :]

地方公共财政收入                            19.19
地区债务率                            0.530361
地区偿债率                            0.002539
地区财政自给率                          0.069208
地区财政依存度                          0.036484
地区财政收入质量                         0.341332
地区生产总值                             105.17
人均地区生产总值                          25477.0
地区第三产业占比                             28.5
人均国内生产总值                          89358.0
地区常住人口                              55.62
地方公共财政收入_2020                      5.4085
地方公共财政收入_2021                      6.4644
地方公共财政收入_2022                        6.59
地方公共财政收入_2023                       19.19
地区生产总值_2020                         95.09
地区生产总值_2021                         100.4
地区生产总值_2022                        100.77
地区生产总值_2023                        105.17
地区常住人口_2020                         26.55
地区常住人口_2021                          26.5
地区常住人口_2022                          26.4
地区常住人口_2023                         55.62
create_time           2024-07-23 0

In [287]:
for column in ['总资产收益率','营业收入','净资产','总资产','营业外收入','利润总额','资本公积','实收资本','货币资金','流动负债',
 'EBITDA利息保障倍数','对外担保','有息负债','资产负债率','短期有息负债',]:
    print(f"Counts for {column}:")
    print( dfrz_qc_index[column].value_counts())
    print()

Counts for 总资产收益率:
总资产收益率
 0.002958    3
 0.007234    2
 0.009539    2
 0.004339    2
-0.004786    2
            ..
 0.010468    1
-0.026959    1
 0.003496    1
 0.004976    1
 0.003077    1
Name: count, Length: 1043, dtype: int64

Counts for 营业收入:
营业收入
3.284092e+09    2
3.070806e+09    2
6.566536e+10    2
6.352052e+09    2
1.341369e+10    1
               ..
1.035845e+10    1
9.542592e+10    1
3.576573e+10    1
1.174966e+09    1
3.270452e+09    1
Name: count, Length: 1067, dtype: int64

Counts for 净资产:
净资产
8.256428e+09    2
5.372023e+10    2
3.551606e+10    2
1.013328e+11    2
1.232357e+11    1
               ..
7.567030e+09    1
4.095651e+10    1
2.622271e+10    1
3.293194e+10    1
3.582070e+10    1
Name: count, Length: 1068, dtype: int64

Counts for 总资产:
总资产
1.968896e+10    2
8.965194e+10    2
1.080747e+11    2
1.788751e+11    2
3.031755e+11    1
               ..
1.297058e+10    1
5.038492e+11    1
1.853951e+11    1
9.319596e+10    1
7.975623e+10    1
Name: count, Length: 1068, dty